# 1. Data Loading 

In [2]:
import pandas as pd

def load_data(path):
    df = pd.read_parquet(path)

    if not isinstance(df.index, pd.DatetimeIndex):
        datetime_col = None
        for col in df.columns:
            if "time" in col.lower() or "date" in col.lower():
                datetime_col = col
                break

        if datetime_col is None:
            raise ValueError("No datetime column found in data.")

        df[datetime_col] = pd.to_datetime(df[datetime_col])
        df = df.set_index(datetime_col)

    df = df.sort_index()
    return df

In [4]:
path = "data_raw\group1"

In [6]:
import os

files = [f for f in os.listdir("../data_raw/group1") if f.endswith(".parquet")]
files

['data1_2023_Q1.parquet',
 'data1_2023_Q3.parquet',
 'data1_2023_Q4.parquet',
 'data1_2024_Q2.parquet',
 'data1_2024_Q4.parquet',
 'data1_2025_Q1.parquet',
 'data1_2025_Q2.parquet']

In [7]:
df = load_data(f"../data_raw/group1/{files[0]}")
df.head()

,NQ,SP
datetime,,
2023-01-02 09:31:00+00:00,NaN,NaN
2023-01-02 09:32:00+00:00,NaN,NaN
2023-01-02 09:33:00+00:00,NaN,NaN
2023-01-02 09:34:00+00:00,NaN,NaN
2023-01-02 09:35:00+00:00,NaN,NaN


## 2. Common Settings (Groups, Costs, Session Rules)

In [ ]:
# ===== Transaction costs & point values =====
CONTRACTS = {
    "SP":  {"cost": 12, "point_value": 50},
    "NQ":  {"cost": 12, "point_value": 20},
    "CAD": {"cost": 10, "point_value": 100000},
    "AUD": {"cost": 10, "point_value": 100000},
    "XAU": {"cost": 15, "point_value": 100},
    "XAG": {"cost": 10, "point_value": 5000},
}

# ===== Group definitions =====
GROUP1_ASSETS = ["SP", "NQ"]              # NYSE session, 1-min data
GROUP2_ASSETS = ["CAD", "AUD", "XAU", "XAG"]  # 24h with break, 5-min data

# ===== Time rules (as strings; we’ll use them later for masks) =====
GROUP1_RULES = {
    "session_start": "09:30",
    "session_end": "16:00",
    "no_trade_start": "09:31",
    "no_trade_end": "09:55",
    "force_exit": "15:40",
    "drop_early_start": "09:31",
    "drop_early_end": "09:40",
    "drop_late_start": "15:51",
    "drop_late_end": "16:00",
}

GROUP2_RULES = {
    "break_start": "17:00",
    "break_end": "18:00",
    "force_exit": "16:50",
    "no_trade_after_break_end": "18:10",
}

ANNUALIZATION_DAYS = 252

In [2]:
print("Group1 assets:", GROUP1_ASSETS)
print("SP cost/point:", CONTRACTS["SP"])
print("Group1 force_exit:", GROUP1_RULES["force_exit"])

Group1 assets: ['SP', 'NQ']
SP cost/point: {'cost': 12, 'point_value': 50}
Group1 force_exit: 15:40
